# Improving Fine-tuned Model using RAG

Code authored by: Shaw Talebi <br>
Article link: https://towardsdatascience.com/how-to-improve-llms-with-rag-abdc132f76ac <br>
Video link: https://youtu.be/Ylz779Op9Pw?si=iOvBETQDrgoK_sO6 <br>
<br>
Colab: https://colab.research.google.com/drive/1peJukr-9E1zCo1iAalbgDPJmNMydvQms?usp=sharing

### imports

In [1]:
!pip3.10 install llama-index-embeddings-huggingface
!pip3.10 install llama-index


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define Settings

In [3]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

LLM is explicitly disabled. Using MockLLM.


### Read and Store Docs into Vector DB

In [4]:
# articles available here: {add GitHub repo}

import os

# List all files in the current directory
print(os.listdir('./'))


# pdf_path = "
# !pip install PyPDF2  # Install PyPDF2 if not already installed

# from PyPDF2 import PdfReader

# book_content = ""
# # Open and read the PDF
# reader = PdfReader(pdf_path)
# for page in reader.pages:
#     book_content += page.extract_text()

# print(book_content)

documents = SimpleDirectoryReader("Downloaded_Books").load_data()

['libextract.py', 'rag_example.ipynb', 'frontend.py', '.gitignore', '.env', '.venv', '.git', 'data', 'testAPI.ipynb', 'output.txt', 'Downloaded_Books']


In [5]:

print(len(documents))

17410


In [6]:
# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)

### Set Up Search Function

In [7]:
# set number of docs to retreive
top_k = 5

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [8]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

### Retrieve Relevant Docs

In [15]:
# query documents
query = input("Enter your prompt:")
response = query_engine.query(query)

In [16]:
# reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
In 
addition, several new readings have been added to keep the text timely and 
culturally relevant.
Overview of the Text
Critical Thinking: A Student’s Introduction  is designed to provide a versatile and 
comprehensive introduction to critical thinking. The book is roughly divided 
into seven major parts:
1. The Fundamentals: Chapters 1–3 introduce students to the basics of 
critical thinking in clear, reader-friendly language.
2. Language: Chapter 4 discusses the uses and pitfalls of language, 
emphasizing the ways in which language can be used to hinder clear, 
effective thinking.
3. Fallacies: Chapters 5 and 6 teach students how to recognize and 
avoid twenty-two of the most common logical

Preface
Nothing is more powerful than reason.
 —Saint Augustine
The first edition of Critical Thinking: A Student’s Introduction  grew out of our 
conviction that a critical thinking text that works—that produces real, measur -
able improvement in students’ critical reasoning skills, a

### Import LLM

In [17]:
!pip3.10 install groq
!pip3.10 install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [18]:
from dotenv import load_dotenv

from groq import Groq

load_dotenv()

# Access the variables
api_key = os.getenv("GROQ_API_KEY")

client = Groq(
    api_key = api_key,
)

### Use LLM

In [19]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]I am a student that is learning from a library

{context}
Please respond to the following comment. Use the context above if it is helpful.

{comment}
[/INST]
"""

In [ ]:
prompt = prompt_template_w_context(context, query)



chat_completion = client.chat.completions.create(
    messages=[
        {
            "role" : "user",
            "content" : prompt
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Based on the provided text, here is a general synopsis of the book "Critical Thinking: A Student's Introduction":

The book is an introduction to critical thinking, designed to be versatile and comprehensive. It is divided into seven major parts, covering topics such as the fundamentals of critical thinking, language, fallacies, and practical applications. The text aims to provide students with a clear and reader-friendly introduction to critical thinking, including abundant exercises to practice their skills. The book emphasizes active, collaborative learning and can be taught in various ways, depending on the instructor's focus.
